In [1]:
import os
import ipywidgets as iw
import matplotlib.pyplot as plt
import numpy as np
import jppy

print(f"jppy version: {jppy.version}")

jppy version: 3.4.1.dev38+g19af23d.d20221023


In [2]:
#Reading the data

input_file = os.path.expandvars("${JPP_DATA}/JOscProbTable.NO.dat")
interpolator = jppy.oscprob.JppyOscProbInterpolator(input_file)

loading oscillation probability table from file /home/yugen96/work/KM3NeT/software/Jpp/Jpp_dev/data//JOscProbTable.NO.dat... OK


In [9]:
#Set the channel

flavour1 = jppy.oscprob.JFlavour_t(12)
flavour2 = jppy.oscprob.JFlavour_t(14)
CParity = jppy.oscprob.JChargeParity_t(1)

channel = jppy.oscprob.JOscChannel(flavour1, flavour2, CParity)
print(jppy.oscprob.JFlavour_t.ELECTRON)

print(channel.Cparity)

xs = np.logspace(1, 5, num=50, base=10)

JFlavour_t.ELECTRON
JChargeParity_t.PARTICLE


In [10]:
xs = np.linspace(10,1000,1000)
#xs = np.logspace(1, 3, num=50, base=10)
parameters = jppy.oscprob.JOscParameters(False)

parameters.set("sinsqTh12", 0.35,
               "sinsqTh13", 0.025, 
               "sinsqTh23", 0.62)

probs = interpolator(parameters,channel,[10],[-1])
print(probs)

[0.00916337]


In [11]:
xs = np.linspace(10,1000,1000)

opts = dict(continuous_update=False)
@iw.interact
def show_pdf(flavour_in=iw.Dropdown(options=[("Electron",12),("Muon",14),("Tau",16)], value=12),
             flavour_out=iw.Dropdown(options=[("Electron",12),("Muon",14),("Tau",16)], value=14),
             CParity=iw.Dropdown(options=[("Particle",1),("Antiparticle",-1)]),
             sinsqTh12=iw.FloatSlider(min=0.2, max=0.35, step=0.01, value=0.304, **opts),
             dM21sq=iw.FloatSlider(min=6.93e-5, max=7.96e-5, step=10e-7, value=7.42e-5,readout_format='.5f', **opts),
             sinsqTh13=iw.FloatSlider(min=0.017, max=0.025, step=0.001, value=0.02246,readout_format='.3f', **opts),
             dM31sq=iw.FloatSlider(min=0.00245,max=0.00269, step=0.00001, value=2.51e-3,readout_format='.4f', **opts),
             sinsqTh23=iw.FloatSlider(min=0.38, max=0.62, step=0.01, value=0.45, **opts),
             deltaCP=iw.FloatSlider(min=0, max=2, step=0.01, value=1.28, **opts)):
    parameters = jppy.oscprob.JOscParameters(sinsqTh12=sinsqTh12,
                                             dM21sq=dM21sq,
                                             sinsqTh13=sinsqTh13,
                                             dM31sq=dM31sq,
                                             sinsqTh23=sinsqTh23,
                                             deltaCP=deltaCP)
    
    channel = jppy.oscprob.JOscChannel(jppy.oscprob.JFlavour_t(flavour_in), 
                                       jppy.oscprob.JFlavour_t(flavour_out), 
                                       jppy.oscprob.JChargeParity_t(CParity))
    
    pdf = lambda E: interpolator(channel,[E],[1])
    
    ys = list(map(pdf, xs))
    y_above = np.max(ys)*1.25
    fig, ax = plt.subplots(figsize=(8,4))
    ax.plot(xs, ys)
    ax.set_xscale('log')
    ax.set_ylim(0, 0.065)
    #ax.set_ylim(0, 1.1)
    plt.grid()
    ax.set_xlabel('E / GeV', fontsize=15)
    ax.set_ylabel('Probability', fontsize=15)

interactive(children=(Dropdown(description='flavour_in', options=(('Electron', 12), ('Muon', 14), ('Tau', 16))…

Tests

In [3]:
#Test if all the probilities add up to 1

xs = np.linspace(10,1000,1000)

opts = dict(continuous_update=False)
@iw.interact
def show_pdf(flavour_in=iw.Dropdown(options=[("Electron",12),("Muon",14),("Tau",16)], value=14),
             CParity=iw.Dropdown(options=[("Particle",1),("Antiparticle",-1)]),
             sinsqTh12=iw.FloatSlider(min=0.2, max=0.35, step=0.01, value=0.304, **opts),
             dM21sq=iw.FloatSlider(min=6.93e-5, max=7.96e-5, step=10e-7, value=7.42e-5,readout_format='.1e', **opts),
             sinsqTh13=iw.FloatSlider(min=0.017, max=0.025, step=0.001, value=0.02246,readout_format='.3f', **opts),
             dM31sq=iw.FloatSlider(min=0.00245,max=0.00269, step=0.00001, value=2.51e-3,readout_format='.4f', **opts),
             sinsqTh23=iw.FloatSlider(min=0.38, max=0.62, step=0.01, value=0.45, **opts),
             deltaCP=iw.FloatSlider(min=0, max=2, step=0.01, value=1.28, **opts)):
    parameters = jppy.oscprob.JOscParameters(sinsqTh12=sinsqTh12,
                                             dM21sq=dM21sq,
                                             sinsqTh13=sinsqTh13,
                                             dM31sq=dM31sq,
                                             sinsqTh23=sinsqTh23,
                                             deltaCP=deltaCP)
    
    interpolator = jppy.oscprob.JOscProbInterpolator8D(table, parameters)
    channel = jppy.oscprob.JOscChannel(jppy.oscprob.JFlavour_t(flavour_in), 
                                       jppy.oscprob.JFlavour_t(12), 
                                       jppy.oscprob.JChargeParity_t(CParity))
    
    pdf1 = lambda E: interpolator(channel,[E],[1])
    ys1 = list(map(pdf1, xs))
    
    interpolator = jppy.oscprob.JOscProbInterpolator8D(table, parameters)
    channel = jppy.oscprob.JOscChannel(jppy.oscprob.JFlavour_t(flavour_in), 
                                       jppy.oscprob.JFlavour_t(14), 
                                       jppy.oscprob.JChargeParity_t(CParity))
    
    pdf2 = lambda E: interpolator(channel,[E],[1])
    ys2 = list(map(pdf2, xs))
    
    channel = jppy.oscprob.JOscChannel(jppy.oscprob.JFlavour_t(flavour_in), 
                                       jppy.oscprob.JFlavour_t(16), 
                                       jppy.oscprob.JChargeParity_t(CParity))
    
    pdf3 = lambda E: interpolator(channel,[E],[1])
    ys3 = list(map(pdf3, xs))
    
    ys_total = np.array(ys1)+np.array(ys2)+np.array(ys3)
    
    fig, ax = plt.subplots(figsize=(8,4))
    ax.plot(xs, ys_total)
    ax.set_xscale('log')
    #ax.set_yscale('log')
    ax.set_ylim(0, 1.1)
    #ax.set_ylim(0, y_above)
    plt.grid()
    ax.set_xlabel('E / GeV', fontsize=15)
    ax.set_ylabel('Probability', fontsize=15)

interactive(children=(Dropdown(description='flavour_in', index=1, options=(('Electron', 12), ('Muon', 14), ('T…

In [13]:
#Met goede y-as

xs = np.linspace(10,1000,1000)

opts = dict(continuous_update=False)
@iw.interact
def choose_channel(flavour_in=iw.Dropdown(options=[("Electron",12),("Muon",14),("Tau",16)], value=12),
                   flavour_out=iw.Dropdown(options=[("Electron",12),("Muon",14),("Tau",16)], value=14),
                   CParity=iw.Dropdown(options=[("Particle",1),("Antiparticle",-1)])):
    channel = jppy.oscprob.JOscChannel(jppy.oscprob.JFlavour_t(flavour_in), 
                                       jppy.oscprob.JFlavour_t(flavour_out), 
                                       jppy.oscprob.JChargeParity_t(CParity))
    
print(channel)

opts = dict(continuous_update=False)
@iw.interact    
def show_pdf(sinsqTh12=iw.FloatSlider(min=0.2, max=0.35, step=0.01, value=0.304, **opts),
             dM21sq=iw.FloatSlider(min=6.93e-5, max=7.96e-5, step=10e-7, value=7.42e-5,readout_format='.5f', **opts),
             sinsqTh13=iw.FloatSlider(min=0.017, max=0.025, step=0.001, value=0.02246,readout_format='.3f', **opts),
             dM31sq=iw.FloatSlider(min=0.00245,max=0.00269, step=0.00001, value=2.51e-3,readout_format='.4f', **opts),
             sinsqTh23=iw.FloatSlider(min=0.38, max=0.62, step=0.01, value=0.45, **opts),
             deltaCP=iw.FloatSlider(min=0, max=2, step=0.01, value=1.28, **opts)):
    parameters = jppy.oscprob.JOscParameters(sinsqTh12=sinsqTh12,
                                             dM21sq=dM21sq,
                                             sinsqTh13=sinsqTh13,
                                             dM31sq=dM31sq,
                                             sinsqTh23=sinsqTh23,
                                             deltaCP=deltaCP)
    interpolator = jppy.oscprob.JOscProbInterpolator8D(table, parameters)
    
    pdf = lambda E: interpolator.__call__(channel,E,1)
    ys = list(map(pdf, xs))
    y_above = np.max(ys)*1.25
    fig, ax = plt.subplots(figsize=(8,4))
    ax.plot(xs, ys)
    ax.set_xscale('log')
    #ax.set_yscale('log')
    ax.set_ylim(0, 0.065)
    #ax.set_ylim(0, y_above)
    plt.grid()
    ax.set_xlabel('E / GeV', fontsize=15)
    ax.set_ylabel('Probability', fontsize=15)



interactive(children=(Dropdown(description='flavour_in', options=(('Electron', 12), ('Muon', 14), ('Tau', 16))…

Cparity=1
in=12
out=14



interactive(children=(FloatSlider(value=0.304, continuous_update=False, description='sinsqTh12', max=0.35, min…

In [40]:
xs=[1,2,3,4]
ys=[2,4,6,8]

def func(a,b):
    return a+b

pdf = lambda a,b: func(a,b)
zs = list(map(pdf,xs,ys))
print(zs)

[3, 6, 9, 12]
